In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import datetime
import scipy.stats as stats
from scipy import sparse
import seaborn as sns
import tensorflow as tf
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, LSTM , Dense,GlobalMaxPooling1D,Flatten
from tensorflow.keras.models import Model
import string
from sklearn.preprocessing import LabelEncoder

In [2]:
dialogueText = pd.read_csv("E:/HANIF/COURSE/Digitalent Kominfo/PROFESSIONAL ACADEMY-MACHINE LEARNING with TENSORFLOW/chatbot/Ubuntu-dialogue-corpus/dialogueText.csv")
dialogueText_196 = pd.read_csv("E:/HANIF/COURSE/Digitalent Kominfo/PROFESSIONAL ACADEMY-MACHINE LEARNING with TENSORFLOW/chatbot/Ubuntu-dialogue-corpus/dialogueText_196.csv")
dialogueText_301 = pd.read_csv("E:/HANIF/COURSE/Digitalent Kominfo/PROFESSIONAL ACADEMY-MACHINE LEARNING with TENSORFLOW/chatbot/Ubuntu-dialogue-corpus/dialogueText_301.csv")

In [3]:
# dialogue text info
dialogueText.info()
dialogueText.describe()
dialogueText.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1038324 entries, 0 to 1038323
Data columns (total 6 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   folder      1038324 non-null  int64 
 1   dialogueID  1038324 non-null  object
 2   date        1038324 non-null  object
 3   from        1038311 non-null  object
 4   to          566035 non-null   object
 5   text        1038237 non-null  object
dtypes: int64(1), object(5)
memory usage: 47.5+ MB


,folder,dialogueID,date,from,to,text
0,3,126125.tsv,2008-04-23T14:55:00.000Z,bad_image,NaN,"Hello folks, please help me a bit with the fol..."
1,3,126125.tsv,2008-04-23T14:56:00.000Z,bad_image,NaN,Did I choose a bad channel? I ask because you ...
2,3,126125.tsv,2008-04-23T14:57:00.000Z,lordleemo,bad_image,the second sentence is better english and we...
3,3,64545.tsv,2009-08-01T06:22:00.000Z,mechtech,NaN,Sock Puppe?t
4,3,64545.tsv,2009-08-01T06:22:00.000Z,mechtech,NaN,WTF?


In [4]:
# dialogue text info
dialogueText_196.info()
dialogueText_196.describe()
dialogueText_196.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9212877 entries, 0 to 9212876
Data columns (total 6 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   folder      int64 
 1   dialogueID  object
 2   date        object
 3   from        object
 4   to          object
 5   text        object
dtypes: int64(1), object(5)
memory usage: 421.7+ MB


,folder,dialogueID,date,from,to,text
9212872,13,3676.tsv,2012-07-07T20:17:00.000Z,MonkeyDust,legolas,= arian
9212873,13,3676.tsv,2012-07-07T20:18:00.000Z,MonkeyDust,legolas,"observation and deduction, dear watson"
9212874,13,16586.tsv,2008-07-25T01:53:00.000Z,linuxfce,NaN,i am trying to install nvidia drivers from the...
9212875,13,16586.tsv,2008-07-25T01:53:00.000Z,linuxfce,NaN,how do i enter runlevel 3? when i try telinit ...
9212876,13,16586.tsv,2008-07-25T01:54:00.000Z,linuxfce,NaN,anyone know how to enter runlevel 3 in ubuntu?


In [5]:
# dialogue text 301 info
dialogueText_301.info()
dialogueText_301.describe()
dialogueText_301.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16587830 entries, 0 to 16587829
Data columns (total 6 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   folder      int64 
 1   dialogueID  object
 2   date        object
 3   from        object
 4   to          object
 5   text        object
dtypes: int64(1), object(5)
memory usage: 759.3+ MB


,folder,dialogueID,date,from,to,text
16587825,32,1783.tsv,2007-11-15T03:38:00.000Z,koyo001,NaN,thanks
16587826,32,1783.tsv,2007-11-15T03:39:00.000Z,koyo001,NaN,does anyone know something
16587827,32,1783.tsv,2007-11-15T03:39:00.000Z,neverblue,NaN,"no, no one knows everything"
16587828,32,1783.tsv,2007-11-15T03:40:00.000Z,koyo001,ikonia,the camera doesnt work
16587829,32,1783.tsv,2007-11-15T03:40:00.000Z,neverblue,koyo001,I believe you missed a post or two while you w...


In [6]:
frames = dialogueText, dialogueText_196, dialogueText_301
data = pd.concat(frames)
data

,folder,dialogueID,date,from,to,text
0,3,126125.tsv,2008-04-23T14:55:00.000Z,bad_image,NaN,"Hello folks, please help me a bit with the fol..."
1,3,126125.tsv,2008-04-23T14:56:00.000Z,bad_image,NaN,Did I choose a bad channel? I ask because you ...
2,3,126125.tsv,2008-04-23T14:57:00.000Z,lordleemo,bad_image,the second sentence is better english and we...
3,3,64545.tsv,2009-08-01T06:22:00.000Z,mechtech,NaN,Sock Puppe?t
4,3,64545.tsv,2009-08-01T06:22:00.000Z,mechtech,NaN,WTF?
...,...,...,...,...,...,...
16587825,32,1783.tsv,2007-11-15T03:38:00.000Z,koyo001,NaN,thanks
16587826,32,1783.tsv,2007-11-15T03:39:00.000Z,koyo001,NaN,does anyone know something
16587827,32,1783.tsv,2007-11-15T03:39:00.000Z,neverblue,NaN,"no, no one knows everything"
16587828,32,1783.tsv,2007-11-15T03:40:00.000Z,koyo001,ikonia,the camera doesnt work


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26839031 entries, 0 to 16587829
Data columns (total 6 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   folder      int64 
 1   dialogueID  object
 2   date        object
 3   from        object
 4   to          object
 5   text        object
dtypes: int64(1), object(5)
memory usage: 1.4+ GB


In [8]:
data.to_csv('E:/HANIF/COURSE/Digitalent Kominfo/PROFESSIONAL ACADEMY-MACHINE LEARNING with TENSORFLOW/chatbot/Ubuntu-dialogue-corpus/chatbot.csv')

In [2]:
chatbot = pd.read_csv("E:/HANIF/COURSE/Digitalent Kominfo/PROFESSIONAL ACADEMY-MACHINE LEARNING with TENSORFLOW/chatbot/Ubuntu-dialogue-corpus/chatbot.csv")
chatbot

,Unnamed: 0,folder,dialogueID,date,from,to,text
0,0,3,126125.tsv,2008-04-23T14:55:00.000Z,bad_image,NaN,"Hello folks, please help me a bit with the fol..."
1,1,3,126125.tsv,2008-04-23T14:56:00.000Z,bad_image,NaN,Did I choose a bad channel? I ask because you ...
2,2,3,126125.tsv,2008-04-23T14:57:00.000Z,lordleemo,bad_image,the second sentence is better english and we...
3,3,3,64545.tsv,2009-08-01T06:22:00.000Z,mechtech,NaN,Sock Puppe?t
4,4,3,64545.tsv,2009-08-01T06:22:00.000Z,mechtech,NaN,WTF?
...,...,...,...,...,...,...,...
26839026,16587825,32,1783.tsv,2007-11-15T03:38:00.000Z,koyo001,NaN,thanks
26839027,16587826,32,1783.tsv,2007-11-15T03:39:00.000Z,koyo001,NaN,does anyone know something
26839028,16587827,32,1783.tsv,2007-11-15T03:39:00.000Z,neverblue,NaN,"no, no one knows everything"
26839029,16587828,32,1783.tsv,2007-11-15T03:40:00.000Z,koyo001,ikonia,the camera doesnt work


In [3]:
chatbot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26839031 entries, 0 to 26839030
Data columns (total 7 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   Unnamed: 0  int64 
 1   folder      int64 
 2   dialogueID  object
 3   date        object
 4   from        object
 5   to          object
 6   text        object
dtypes: int64(2), object(5)
memory usage: 1.4+ GB


In [8]:
del chatbot['Unnamed: 0']

In [14]:
chatbot.head()

,folder,dialogueID,date,from,to,text
0,3,126125.tsv,2008-04-23T14:55:00.000Z,bad_image,NaN,"Hello folks, please help me a bit with the fol..."
1,3,126125.tsv,2008-04-23T14:56:00.000Z,bad_image,NaN,Did I choose a bad channel? I ask because you ...
2,3,126125.tsv,2008-04-23T14:57:00.000Z,lordleemo,bad_image,the second sentence is better english and we...
3,3,64545.tsv,2009-08-01T06:22:00.000Z,mechtech,NaN,Sock Puppe?t
4,3,64545.tsv,2009-08-01T06:22:00.000Z,mechtech,NaN,WTF?


In [12]:
#list(chatbot.text)

In [16]:
chatbot['text'] = chatbot['text'].astype('str')

In [ ]:
#removing punctuations
chatbot['text'] = chatbot['text'].apply(lambda wrd:[ltrs.lower() for ltrs in wrd if ltrs not in string.punctuation])
chatbot['text'] = chatbot['text'].apply(lambda wrd: ''.join(wrd))

In [ ]:
#tokenize the data
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(chatbot['text'])
train = tokenizer.texts_to_sequences(chatbot['text'])

In [ ]:
#apply padding
x_train = pad_sequences(train)

In [ ]:
#encoding the outputs
le = LabelEncoder()
y_train = le.fit_transform(chatbot['from','to'])